In [7]:
import pandas as pd
#import mrmr
import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_regression
import tensorflow as tf
from tensorflow import keras
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier

Goal: combine thousands of prediction files to decrease the log error of a binary classification. Managed to decrease the error from 0.63 to ~ 0.529. Data is available at https://www.kaggle.com/competitions/tabular-playground-series-nov-2022.

In [8]:
labels = pd.read_csv(r"D:\Data for python\datasets\tabular-playground-series-nov-2022\train_labels.csv")
submissions = [os.path.join(r'D:\Data for python\datasets\tabular-playground-series-nov-2022\submission_files', file) for file in os.listdir(r'D:\Data for python\datasets\tabular-playground-series-nov-2022\submission_files')]
sample_sub = pd.read_csv(r"D:\Data for python\datasets\tabular-playground-series-nov-2022\sample_submission.csv")



In [9]:
def kaggle_submission(trained_model, test_data, columns: list[str] = ["id", "pred"],savename="my_predictions", neural=False, round=False):
  SAVING_FOLDER = os.path.join(os.path.join(
    os.environ['USERPROFILE']), 'Desktop')
  predictions = trained_model.predict(test_data)
  if neural:
    predictions = [pred for [pred] in predictions]
  if round:
    predictions = [round(num) for num in predictions]
  indices = range(20000, 40000)
  sub_df = pd.DataFrame(zip(indices, predictions) , columns=columns)
  with open(f"{SAVING_FOLDER}/{savename}.csv", 'w', encoding = 'utf-8-sig') as f:
    sub_df.to_csv(f, index=False)  

In [10]:
df0 = pd.read_csv(submissions[0])

In [11]:
df0

,id,pred
0,0,0.709336
1,1,0.452988
2,2,0.675462
3,3,0.481046
4,4,0.957339
...,...,...
39995,39995,0.382515
39996,39996,0.352498
39997,39997,0.577554
39998,39998,0.712353


In [25]:
NUM_SUBMISSIONS = 300

df = df0.copy()

for i, sub in enumerate(submissions[1:NUM_SUBMISSIONS]):
    df = df.join(pd.read_csv(sub).pred, rsuffix=f"{i}")

In [26]:
df = df.drop("id", axis=1)
x , test = df.iloc[:20000], df.iloc[20000:]
y = labels.label

MRMR algorithm from https://towardsdatascience.com/mrmr-explained-exactly-how-you-wished-someone-explained-to-you-9cf4ed27458b

In [27]:
# inputs:
#    X: pandas.DataFrame, features
#    y: pandas.Series, target variable
#    K: number of features to select
X = x
K = 100


# compute F-statistics and initialize correlation matrix
F = pd.Series(f_regression(X, y)[0], index = X.columns)
corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

# initialize list of selected features and list of excluded features
selected = []
not_selected = X.columns.to_list()

# repeat K times
for i in range(K):
  
    # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
    if i > 0:
        last_selected = selected[-1]
        corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
        
    # compute FCQ score for all the (currently) excluded features (this is Formula 2)
    score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
    
    # find best feature, add it to selected and remove it from not_selected
    best = score.index[score.argmax()]
    selected.append(best)
    not_selected.remove(best)


x = x.drop(not_selected, axis=1)
test = test.drop(not_selected, axis=1)


In [28]:
SPLIT_CUT = 16000

x_train_full, x_test, y_train_full, y_test =  train_test_split(x, y, test_size=0.05)          
x_train, x_valid =  x_train_full[:SPLIT_CUT], x_train_full[SPLIT_CUT:]   
y_train, y_valid =  y_train_full[:SPLIT_CUT], y_train_full[SPLIT_CUT:]


In [29]:
#xgb = XGBClassifier(n_estimators= 200, learning_rate = 0.2)
#forest = RandomForestClassifier(n_estimators=200)
#xgb.fit(x_train.to_numpy(), y_train.to_numpy())
#preds = xgb.predict(x_test.to_numpy())
#log_loss(y_pred = preds, y_true=y_test.to_numpy())

In [30]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor="loss")

sample_to_normalize = x_train
norm_layer = keras.layers.Normalization(axis=1)
norm_layer.adapt(tf.constant(sample_to_normalize))

neural = keras.models.Sequential([
norm_layer,
keras.layers.Dense(64, activation="relu",input_shape=x_train.shape[1:]),
keras.layers.Dropout(rate=0.1),
keras.layers.Dense(64,  "relu"),
keras.layers.Dropout(rate=0.1),
keras.layers.Dense(64,  "relu"),
keras.layers.Dense(1, activation=None),
])

optimizer3 = keras.optimizers.SGD(learning_rate=0.003, nesterov=True) #keras.optimizers.RMSprop(lr=0.001) 
optimizer = keras.optimizers.Nadam(learning_rate=0.001)
optimizer2 = keras.optimizers.SGD(learning_rate= 0.01)
loss = tf.keras.losses.MeanSquaredError()
neural.compile(loss="mean_squared_error", optimizer= optimizer3)

history = neural.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=200, callbacks=[early_stopping_cb])

Epoch 1/200
500/500 [==============================] - 4s 6ms/step - loss: 0.2169 - val_loss: 0.1853
Epoch 2/200
500/500 [==============================] - 2s 5ms/step - loss: 0.1912 - val_loss: 0.1843
Epoch 3/200
500/500 [==============================] - 3s 6ms/step - loss: 0.1877 - val_loss: 0.1836
Epoch 4/200
500/500 [==============================] - 3s 5ms/step - loss: 0.1868 - val_loss: 0.1834
Epoch 5/200
500/500 [==============================] - 3s 6ms/step - loss: 0.1848 - val_loss: 0.1833
Epoch 6/200
500/500 [==============================] - 3s 5ms/step - loss: 0.1843 - val_loss: 0.1839
Epoch 7/200
500/500 [==============================] - 3s 6ms/step - loss: 0.1838 - val_loss: 0.1835
Epoch 8/200
500/500 [==============================] - 3s 6ms/step - loss: 0.1832 - val_loss: 0.1849
Epoch 9/200
500/500 [==============================] - 2s 5ms/step - loss: 0.1829 - val_loss: 0.1846
Epoch 10/200
500/500 [==============================] - 2s 5ms/step - loss: 0.1829 - val_lo

In [31]:

kaggle_submission(neural, test, savename="300 to 100 mrmr col neural 64n", neural = True)

625/625 [==============================] - 2s 3ms/step


In [32]:
pred = neural.predict(x_test)
y_true = y_test.to_numpy()
y_pred = np.array([val[0] for val in pred])
y_pred_rounded = [val[0].round() for val in pred]
loss = metrics.log_loss( y_true , y_pred)
acc_score = metrics.accuracy_score( y_true, y_pred_rounded)

print(f"log loss: {loss}   accuracy score: {acc_score}")

32/32 [==============================] - 0s 3ms/step
log loss: nan   accuracy score: 0.72


c:\Users\flavi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2442: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
c:\Users\flavi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2442: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
